# Hash Tables

A hash table is a collection that maps keys to values.  Python's `dict` is an implementation of a hash table.

For the course project, you will be implementing a hash table without using `dict`.

### Hash Table Performance

| Operation | Average | Worst Case | 
| --------- | ---- | ---------- |
| lookup    | O(1) | O(n) |
| insert    | O(1) | O(n) |
| delete    | O(1) | O(n) |

Note: These are average case, as we'll see, depending on implementation, worst case can be much worse.

A key property for hash tables is that we **do not need to linearly search through them for our data**.

If you find yourself scanning every element in a hash table, you're doing something wrong.

### Example

Let's first model a simple hashtable with fixed capacity of 10.  For simplicity we'll stick to string keys.

```
[    ][    ][    ][    ][    ][    ][    ][    ][    ][    ]
```

When we get a key-value pair, we need to assign it a bucket.

How can we write a function that takes a string and assigns it to a bucket?

1. Turn string into a number. **Hash Function**
2. Take (number % capacity)

In [11]:
def strhash(key):
    # ord converts a character to it's numeric representation
    #   ord("A") == 65
    #   ord("z") == 122
    # etc.
    return sum(ord(letter) for letter in key)

In [12]:
strhash("bear")

410

In [13]:
strhash("fox")

333

In [14]:
class Hashtable:
    def __init__(self, capacity=10):
        self._table = [None] * capacity
        self.capacity = capacity

    def __setitem__(self, key, value):
        index = strhash(key) % self.capacity
        self._table[index] = value

    def __getitem__(self, key):
        index = strhash(key) % self.capacity
        value = self._table[index]
        if value:
            return value
        else:
            raise KeyError(key)

    def display(self):
        print(f"Capacity = {self.capacity}")
        for idx, elem in enumerate(self._table):
            if elem:
                print(f"{idx}: {elem}")

In [15]:
h = Hashtable()
h["bear"] = 3
h["fox"] = 12
h.display()

Capacity = 10
0: 3
3: 12


In [16]:
print(h["bear"])

3


In [17]:

strhash("bear") == strhash("been")  # different word, same hash!

True

In [19]:
h = Hashtable()
h["bear"] = 33
h["been"] = 12
h.display()

Capacity = 10
0: 12


### Linear Probing

One solution is to just walk forward in the storage list, until we find an empty space.

Either way, we'll need to start storing the key as well.   Let's revise our class:

In [6]:
class Hashtable:
    def __init__(self, capacity=100):
        self._table = [None] * 100
        self.capacity = capacity

    def __setitem__(self, key, value):
        index = strhash(key) % self.capacity
        while self._table[index] is not None:
            index += 1
            # Handling wrap-around omitted for brevity

        # we now store the key and value
        self._table[index] = (key, value)

    def __getitem__(self, key):
        index = strhash(key) % self.capacity

        # walk forward until we either reach the item or an empty space
        while self._table[index] is not None:
            if self._table[index][0] == key:
                return self._table[index][1]
            index += 1
            # Handling wrap-around omitted for brevity

        # if code got here, the item wasn't in the list
        raise KeyError(key)

    def display(self):
        print(f"Capacity = {self.capacity}")
        for idx, elem in enumerate(self._table):
            if elem:
                print(f"{idx}: {elem}")

In [20]:
h = Hashtable()
h["bear"] = 33
h["been"] = 12
h.display()

Capacity = 10
0: 12


In [21]:
print(h["bear"])
print(h["been"])

12
12


### Separate Chaining

Another solution is to use a linked list to store multiple items in the same bucket.  One could imagine each bucket being the head of a linked list where items that hash to that value are stored linearly.

This is the approach we're asking you to use for the course project.

**What happens if we have a lot of items in the same bucket?**

### Better Hash Function

Ideally a hash function will evenly distribute values across the collection.

A common pattern is to use a polynomial hash function.

$$h(x_0, ..., x_n) = (\sum_{i=0}^{k-1}{c_ip^i})\mod{m}$$

Where:
- $x_0...x_i $ is the sequence
- $k = len(x)$
- $c_i$ is the numeric value of the character $x_i$ ($ord(x$ in Python)
- $p$ is an arbitrary constant.
- and $m$ is the size of the collection.

```python
    def _hash(self, key):
        """
        This method takes in a string and returns 
        an integer value between 0 and self.capacity.

        This particular hash function uses 
        Horner's rule to compute a large polynomial.
        """
        val = 0
        for letter in key:
            val = self.P_CONSTANT * val + ord(letter)
        return val % self.capacity
```

### Questions

* Linear probing vs. separate chaining? Other approaches?
* What if our hash function wasn't reliable?

### Rehashing

As we add more items to our hash table, we'll eventually run out of space.  We can increase the capacity of our table, but we'll need to rehash all of our existing items.

Because which bucket we choose depends on `hash(item) % capacity` items would end up in different buckets if we change capacity.

This means when we resize, we need to rehash all of our items.

A common pattern is to double capacity when the table is ~50% full.